## INSTALLING THE NECESSARY DEPENDENCIES:

In [1]:
##(install and restart runtime)
#!pip install --upgrade accelerate
#!pip install transformers[sentencepiece]

In [2]:
import torch
import numpy as np
import pandas as pd

!pip install transformers datasets huggingface_hub tensorboard==2.11
!sudo apt-get install git-lfs --yes
import os

from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments, AutoModelForSequenceClassification, Trainer

torch.__version__


from huggingface_hub import HfFolder, notebook_login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.1
    Uninstalling tensorboard-data-server-0.7.1:
      Successfully uninstalled tensorboard-data-server-0.7.1
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.0.0
    Uninstalling google-auth-oauthlib-1.0.0:
      Successfully uninstalled google-auth-oauthlib-1.0.0
  Attempt

In [3]:
notebook_login()

In [4]:
repository_id = "Geee05/overlap2"

In [5]:
dataset = pd.read_csv('/content/All_transcript_train.csv')
tdataset= pd.read_csv('/content/All_transcript_test_new.csv')

## DATA PRE-PROCESSING

In [6]:
dataset.rename(columns = {'Class':'labels'}, inplace = True)
tdataset.rename(columns = {'Group':'labels'}, inplace = True)

In [7]:
dataset.replace({'labels':{"HC":0,"DP":1}},inplace=True)
tdataset.replace({'labels':{"HC":0,"DP":1}},inplace=True)

In [8]:
dataset['labels'].value_counts()

0    126
1     37
Name: labels, dtype: int64

In [9]:
#resample data to deal with class imbalance
from sklearn.utils import resample
df_1=dataset[dataset['labels']==1]
df_2=dataset[dataset['labels']!=1]
df_upsampled=resample(df_1,random_state=42,n_samples=126,replace=True)
df=pd.concat([df_upsampled,df_2])
df.reset_index(inplace=True)

In [10]:
#creating Datasets
train = Dataset.from_pandas(df, preserve_index=False)
test = Dataset.from_pandas(tdataset, preserve_index=False)

print(train)
print(test)

Dataset({
    features: ['index', 'Unnamed: 0', 'File', 'Text', 'labels'],
    num_rows: 252
})
Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'File', 'Text', 'labels'],
    num_rows: 56
})


In [11]:
leng=[]
for i in range (0,len(train['Text'])):
  leng.append(len(train['Text'][i]))

print(min(leng))
print(max(leng))

1362
21064


### Function Definition:

In [12]:
#SEGMENT TEXTS
def segment_function(example):
  seg=example['Text']
  res=[]
  n=10000

  for iter in range (0,len(seg)):
    for i in range (0,len(seg[iter])):
      j=i*5000

      if len(seg[iter])<10000:
          part=(seg[iter])
          res.append(part)
          break
      elif j<=len(seg[iter]):
          part=(seg[iter][j:j+n])
          res.append(part)
      else:
        break

  return res

#TOKENIZE
tokenizer = AutoTokenizer.from_pretrained('gooohjy/suicidal-electra')
def tokenize_function(example):
  return tokenizer(example["Text"], padding='max_length',max_length=512,truncation=True,add_special_tokens = True)

In [13]:
#LABELS CORR. TO SEGMENTS
label=[]
for i in range (0,len(train)):
  if (len(train['Text'][i])) <10000:
    label.append(train['labels'][i])
  else:
    x = ((len(train['Text'][i]))//5000) + 1
    for j in range (0,x):
      label.append(train['labels'][i])



label_t=[]
for i in range (0,len(test)):
  if (len(test['Text'][i])) <10000:
    label_t.append(test['labels'][i])
  else:
    x = ((len(test['Text'][i]))//5000) + 1
    for j in range (0,x):
      label_t.append(test['labels'][i])

#### Segmenting Train Data

In [14]:
ip_seg=segment_function(train)

In [15]:
len(ip_seg)

362

In [16]:
#preparing dataframe for segmented texts & corresponding labels
train_data=pd.DataFrame(ip_seg,columns=['Text'])
train_data['labels']=label
train_data.head(2)

,Text,labels
0,okay hi I'm Ellie thanks for coming in today ...,1
1,and please are you okay yes I'm alright with ...,1


### Tokenizing Train Data

In [17]:
trainset = Dataset.from_pandas(train_data, preserve_index=False)
trainset

Dataset({
    features: ['Text', 'labels'],
    num_rows: 362
})

In [18]:
tokenized_train_dataset = trainset.map(tokenize_function, batched=True)
tokenized_train_dataset

Map:   0%|          | 0/362 [00:00<?, ? examples/s]

Dataset({
    features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 362
})

#### Segmenting Test Data

In [19]:
ip_seg_t=segment_function(test)

test_data=pd.DataFrame(ip_seg_t,columns=['Text'])
test_data['labels']=label_t
test_data.head(2)

,Text,labels
0,okay there she is coming to go ahead and Shrin...,0
1,this is super need I like this me either at a...,1


### Tokenizing Test Data

In [20]:
testset = Dataset.from_pandas(test_data, preserve_index=False)
testset

Dataset({
    features: ['Text', 'labels'],
    num_rows: 71
})

In [21]:
tokenized_test_dataset = testset.map(tokenize_function, batched=True)
tokenized_test_dataset

Map:   0%|          | 0/71 [00:00<?, ? examples/s]

Dataset({
    features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 71
})

### Review

In [22]:
#tokenizer.model_max_length =512

In [23]:
print(len(label))
print(len(label_t))


362
71


In [24]:
print(len(ip_seg))
print(len(ip_seg_t))


362
71


In [25]:
type(tokenized_train_dataset)

datasets.arrow_dataset.Dataset

## TRAINING

In [26]:
# Set dataset format using set_format() function to specify the dataset format, making it compatible with PyTorch.
tokenized_train_dataset.set_format("torch", columns=['input_ids','attention_mask','labels'])
tokenized_test_dataset.set_format("torch", columns=['input_ids','attention_mask','labels'])

In [27]:
os.environ["WANDB_DISABLED"] = "true"

In [29]:
from sklearn.metrics import classification_report

training_args = TrainingArguments(evaluation_strategy="epoch",num_train_epochs=35,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    output_dir=repository_id,
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token())   # default arguments for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained('gooohjy/suicidal-electra', num_labels=2) #,ignore_mismatched_sizes=True
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    #probabilities = softmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}
    print((classification_report(labels,predictions)))

trainer = Trainer(   # specifying trainer class
    model,
    training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


Cloning https://huggingface.co/Geee05/overlap2 into local empty directory.


Download file pytorch_model.bin:   0%|          | 7.37k/418M [00:00<?, ?B/s]

Download file logs/events.out.tfevents.1689070768.e36198f2fb2b.1201.0: 100%|##########| 4.85k/4.85k [00:00<?, …

Clean file logs/events.out.tfevents.1689070768.e36198f2fb2b.1201.0:  21%|##        | 1.00k/4.85k [00:00<?, ?B/…

Download file training_args.bin: 100%|##########| 3.87k/3.87k [00:00<?, ?B/s]

...

Download file logs/events.out.tfevents.1689087131.b0e5d2dfe1cc.2760.0: 100%|##########| 8.36k/8.36k [00:00<?, …

Clean file logs/events.out.tfevents.1689087131.b0e5d2dfe1cc.2760.0:  12%|#1        | 1.00k/8.36k [00:00<?, ?B/…

Download file logs/events.out.tfevents.1689097370.b0e5d2dfe1cc.45362.0: 100%|##########| 5.00k/5.00k [00:00<?,…

Clean file logs/events.out.tfevents.1689097370.b0e5d2dfe1cc.45362.0:  20%|#9        | 1.00k/5.00k [00:00<?, ?B…

Clean file pytorch_model.bin:   0%|          | 1.00k/418M [00:00<?, ?B/s]

In [30]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.654000,1.586126,0.563380,0.474576
2,0.704500,1.007282,0.633803,0.500000
3,0.512900,0.707374,0.633803,0.480000
4,0.352500,0.904701,0.591549,0.452830
5,0.067000,1.225495,0.718310,0.523810
6,0.015400,1.457521,0.704225,0.461538
7,0.001000,1.924525,0.661972,0.500000
8,0.189900,2.893273,0.591549,0.472727
9,0.026800,2.168076,0.718310,0.523810
10,0.091500,2.586873,0.661972,0.500000


<ipython-input-29-a51742d4a3cf>:25: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


TrainOutput(global_step=1610, training_loss=0.15048356936152327, metrics={'train_runtime': 2595.7313, 'train_samples_per_second': 4.881, 'train_steps_per_second': 0.62, 'total_flos': 3333617071411200.0, 'train_loss': 0.15048356936152327, 'epoch': 35.0})

In [31]:
trainer.state.best_model_checkpoint

'Geee05/overlap2/checkpoint-874'

In [32]:
# Save our tokenizer and create a model card
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub()

Upload file logs/events.out.tfevents.1689153678.e2807d26ba48.616.0:   0%|          | 1.00/41.6k [00:00<?, ?B/s…

To https://huggingface.co/Geee05/overlap2
   e238cd0..23ce8b2  main -> main

   e238cd0..23ce8b2  main -> main



'https://huggingface.co/Geee05/overlap2/commit/23ce8b20471540b70b7a7dafae5fc6c33ab91d0f'

In [ ]:
#model=trainer.save_model

## PREDICTION

In [33]:
predict=trainer.predict(tokenized_test_dataset)

In [34]:
predict[0]

array([[ 4.5102367, -4.756154 ],
       [ 5.370767 , -5.528025 ],
       [ 5.393155 , -5.5940747],
       [ 5.3944745, -5.57     ],
       [ 5.4103923, -5.589987 ],
       [ 1.5504621, -1.8947773],
       [ 5.351244 , -5.5385427],
       [ 4.896025 , -5.0597005],
       [ 5.4170995, -5.532297 ],
       [ 5.39185  , -5.5694017],
       [ 5.042748 , -5.276358 ],
       [ 4.546623 , -4.765131 ],
       [ 5.3242106, -5.5139723],
       [ 4.65798  , -4.92798  ],
       [ 5.3719625, -5.509377 ],
       [-6.016693 ,  5.3892717],
       [ 5.3193884, -5.5297756],
       [ 4.792147 , -4.9966984],
       [ 5.3019366, -5.422876 ],
       [ 2.5288212, -2.8624713],
       [ 5.334479 , -5.47338  ],
       [ 5.3924623, -5.527839 ],
       [ 5.3492656, -5.4797044],
       [-1.493918 ,  1.1312596],
       [-2.8229523,  2.3963692],
       [ 5.354909 , -5.5307384],
       [ 4.8217993, -5.059022 ],
       [-5.781203 ,  5.158772 ],
       [ 5.3408756, -5.5533905],
       [ 5.2582364, -5.4476757],
       [-3

## VALIDATION

In [35]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Geee05/overlap")
model_electra = AutoModelForSequenceClassification.from_pretrained("Geee05/overlap")

In [36]:
vdataset= pd.read_csv('/content/All_transcript_val_new.csv')

In [37]:
vdataset.head(2)

,Unnamed: 0.1,Unnamed: 0,File,Text,Group
0,0,0,600_AUDIO,okay there she is coming to go ahead and Shrin...,HC
1,1,1,602_AUDIO,this is super need I like this me either at a...,DP


In [38]:
vdataset.rename(columns = {'Group':'labels'}, inplace = True)
vdataset.replace({'labels':{"HC":0,"DP":1}},inplace=True)
vdataset['labels'].value_counts()

0    39
1    17
Name: labels, dtype: int64

In [39]:
val = Dataset.from_pandas(vdataset, preserve_index=False)
print(val)

Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'File', 'Text', 'labels'],
    num_rows: 56
})


In [40]:
#LABELS CORR. TO SEGMENTS

label_v=[]
for i in range (0,len(val)):
  if (len(val['Text'][i])) <10000:
    label_v.append(val['labels'][i])
  else:
    x = ((len(val['Text'][i]))//5000) + 1
    for j in range (0,x):
      label_v.append(val['labels'][i])

In [41]:
ip_seg_v=segment_function(val)

In [42]:
print(len(label_v))
print(len(ip_seg_v))

71
71


In [43]:
ip_seg_v=segment_function(val)

val_data=pd.DataFrame(ip_seg_v,columns=['Text'])
val_data['labels']=label_v
val_data.head(2)

,Text,labels
0,okay there she is coming to go ahead and Shrin...,0
1,this is super need I like this me either at a...,1


In [44]:
valset = Dataset.from_pandas(val_data, preserve_index=False)
valset

Dataset({
    features: ['Text', 'labels'],
    num_rows: 71
})

In [45]:
tokenized_val_dataset = valset.map(tokenize_function, batched=True)
tokenized_val_dataset

Map:   0%|          | 0/71 [00:00<?, ? examples/s]

Dataset({
    features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 71
})

In [46]:
tokenized_val_dataset.set_format("torch", columns=['input_ids','attention_mask','labels'])

In [47]:
tokenized_val_dataset

Dataset({
    features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 71
})

In [48]:
tokenized_val_dataset['input_ids'].shape

torch.Size([71, 512])

In [49]:
logits=[]
labels=[]
for i in range (0,len(tokenized_val_dataset['Text'])):

  inputs = tokenizer(tokenized_val_dataset['Text'][i],padding='max_length',max_length=512,truncation=True,add_special_tokens = True, return_tensors="pt")
  with torch.no_grad():
    outputs = model_electra(**inputs)
    logits_l = model_electra(**inputs).logits
    #print(logits)
    logits.append(logits_l)
    labels_i = logits_l.argmax().item()
  labels.append(labels_i)

  #print(labels)

    #print(outputs)
  #print(np.array(inputs['input_ids']).shape)
  #print(np.array(inputs['token_type_ids']).shape)
  #print(np.array(inputs['attention_mask']).shape)
  #print(**inputs.shape)

  #last_hidden_states = outputs.last_hidden_state
  #result.append(outputs)

In [50]:
len(labels)

71

In [51]:
len(logits)

71

In [52]:
logits

[tensor([[-0.2904,  0.2225]]),
 tensor([[ 4.6095, -4.6444]]),
 tensor([[ 4.7588, -4.8537]]),
 tensor([[ 4.7731, -4.8482]]),
 tensor([[ 5.0388, -5.1447]]),
 tensor([[ 3.5424, -3.5500]]),
 tensor([[ 5.0549, -5.1543]]),
 tensor([[ 3.0726, -3.0641]]),
 tensor([[ 4.9509, -5.0066]]),
 tensor([[ 4.9254, -5.0034]]),
 tensor([[ 4.3699, -4.4225]]),
 tensor([[ 3.4817, -3.4906]]),
 tensor([[ 4.6348, -4.6966]]),
 tensor([[ 0.4240, -0.5027]]),
 tensor([[ 4.6962, -4.7387]]),
 tensor([[-4.9565,  4.4561]]),
 tensor([[ 4.6513, -4.7107]]),
 tensor([[ 4.0101, -4.0449]]),
 tensor([[ 4.1006, -4.0973]]),
 tensor([[-4.1458,  3.7066]]),
 tensor([[ 4.6119, -4.6484]]),
 tensor([[ 4.8490, -4.9112]]),
 tensor([[ 4.0216, -4.0048]]),
 tensor([[-4.3132,  3.9028]]),
 tensor([[-3.8877,  3.5614]]),
 tensor([[ 4.6414, -4.7118]]),
 tensor([[ 3.9913, -4.0199]]),
 tensor([[-2.9366,  2.6586]]),
 tensor([[ 4.7997, -4.8814]]),
 tensor([[ 3.9065, -3.9110]]),
 tensor([[ 3.2822, -3.2865]]),
 tensor([[ 4.3038, -4.3698]]),
 tensor(

In [53]:
true_labels=[]
for i in range (0,len(tokenized_val_dataset['labels'])):
  true_labels.append(tokenized_val_dataset['labels'][i])


In [54]:
len(true_labels)

71

In [55]:
from sklearn.metrics import classification_report

print(classification_report(true_labels,labels))

              precision    recall  f1-score   support

           0       0.79      0.86      0.82        49
           1       0.61      0.50      0.55        22

    accuracy                           0.75        71
   macro avg       0.70      0.68      0.69        71
weighted avg       0.74      0.75      0.74        71



## SAVING THE RESULTS

In [56]:
new_df=pd.DataFrame(predict[0], columns=["logit1","logit1"])
#new_df['pred_label']=predict[1]

In [57]:
pred_labels=predict[1]

In [58]:
type(predict)

transformers.trainer_utils.PredictionOutput

In [59]:
speaker=[]
for i in range (0,len(test)):
  if (len(test['Text'][i])) <10000:
    speaker.append(test['File'][i])
  else:
    x = ((len(test['Text'][i]))//5000) + 1
    for j in range (0,x):
      speaker.append(test['File'][i])

In [61]:
ref_df=pd.DataFrame(predict[0], columns=["logit1","logit2"])
ref_df['subject']=speaker
ref_df['pred_labels']=predict[1]
ref_df

,logit1,logit2,subject,pred_labels
0,4.510237,-4.756154,600_AUDIO,0
1,5.370767,-5.528025,602_AUDIO,1
2,5.393155,-5.594075,604_AUDIO,1
3,5.394475,-5.570000,605_AUDIO,0
4,5.410392,-5.589987,606_AUDIO,0
...,...,...,...,...
66,5.116223,-5.218317,712_AUDIO,0
67,5.172581,-5.326568,715_AUDIO,0
68,4.836604,-5.057820,716_AUDIO,1
69,5.330131,-5.526917,717_AUDIO,0


In [62]:
len(speaker)

71

In [63]:
new_df['subject']=speaker
new_df.head()

,logit1,logit1,subject
0,4.510237,-4.756154,600_AUDIO
1,5.370767,-5.528025,602_AUDIO
2,5.393155,-5.594075,604_AUDIO
3,5.394475,-5.570000,605_AUDIO
4,5.410392,-5.589987,606_AUDIO


In [66]:
#logits datasheet
new_df.to_csv('new_df(10000).csv')

In [65]:
#logits and predicted labels datasheet
ref_df.to_csv('ref_df.csv')

In [ ]:
import itertools
subject = new_df['subject']
# empty list
result = []

iterator = itertools.groupby(subject, lambda string: string.split('_')[0])

for element, group in iterator:

   result.append(list(group))

for i in range (0,len(new_df)):
  df=new_df[new_df['subject'].isin(result(i))]